# PyTorch Verification

In [1]:
import torch

print(torch.__version__)

2.1.0


In [2]:
# Is MPS even available? macOS 12.3+
print(torch.backends.mps.is_available())

# Was the current version of PyTorch built with MPS activated?
print(torch.backends.mps.is_built())

True
True


In [3]:
import math

dtype = torch.float
device = torch.device("mps")

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
 

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 200.4646759033203
199 139.99082946777344
299 98.70269012451172
399 70.48092651367188
499 51.16817855834961
599 37.93696975708008
699 28.861812591552734
799 22.63014793395996
899 18.346210479736328
999 15.397920608520508
1099 13.366617202758789
1199 11.965546607971191
1299 10.998146057128906
1399 10.32947063446045
1499 9.866812705993652
1599 9.546354293823242
1699 9.324193954467773
1799 9.170026779174805
1899 9.062941551208496
1999 8.988494873046875
Result: y = -0.012267143465578556 + 0.8508265614509583 x + 0.0021162854973226786 x^2 + -0.09248913824558258 x^3


# TensorFlow Verification

In [4]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
# print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.6.1-arm64-i386-64bit
Tensor Flow Version: 2.14.0

Python 3.9.18 (main, Sep 11 2023, 08:25:10) 
[Clang 14.0.6 ]
Pandas 2.1.1
Scikit-Learn 1.3.0
SciPy 1.11.3
GPU is available


In [5]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=64)

2023-10-29 22:23:00.079899: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-29 22:23:00.079938: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-29 22:23:00.079941: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-29 22:23:00.079986: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-29 22:23:00.080234: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-10-29 22:23:05.112256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2023-10-29 22:23:05.727270: E te

782/782 [==============================] - 120s 145ms/step - loss: 4.5339 - accuracy: 0.0892
